In [1]:
import yaml
import os
os.chdir('..')
print(os.getcwd())

z:\OMOP\omop_etl


In [2]:
%run -i run_etl.py

Preloading drug_exposure (order) ...
preload complete. Elapsed time 00:00:06
Preloading drug_exposure (admin) ...
preload complete. Elapsed time 00:00:06
Loading drug_exposure ...
load_table complete. Elapsed time 00:00:13
5904 rows affected
z:\OMOP\omop_etl


In [3]:
from omop_etl.postproc.load import Loader, truncate
load = Loader('config.yml')
# load.load_person()
# load.preload('measurement','height')

In [5]:
load.load_table('death')

Loading death ...
load_table complete. Elapsed time 00:00:00


'0 rows affected'

In [4]:
# truncate('preload','drug_exposure', load.engine)

In [33]:
f = open('config.yml')
load = yaml.safe_load(f)
# print(preload)
# list(preload['preload']['drug'].keys())

In [34]:
STAGE = {
    'measurement': {
        'res_pip': 'MEASUREMENT_Res_PIP'
    }
}

for t in load['load']:
    STAGE[t]

KeyError: 'person'

In [8]:
from omop_etl.datastore import DataStore, execute
omop = DataStore('config.yml')
t = omop.list_tables(in_schema=['preload'])

In [21]:
# for table in t.Table:
#     execute(f'drop table if exists preload.{table}', omop.engine)

In [2]:
%run -i omop_etl/pullrawdata/pull_raw_data_v2.py

Start time: 2021-01-14 09:51:51.962589

